<center>

<div style="text-align:center"><img src="_static/small_e_logo_cropped.png" width="40%" /></div>
<div style="text-align:center"><img src="_static/pangeo_simple_logo.png" width="175px" /></div>
</center>

Pangeo Tools
===========




A brief overview of the ecosystem of tools used by the [Pangeo Project](http://pangeo.io/).

In [ ]:
%matplotlib inline
import numpy as np
import xarray as xr
import warnings

warnings.simplefilter("ignore", )
np.set_printoptions(precision=4, threshold=5)
xr.set_options(display_width=80)
%config InlineBackend.figure_format = 'retina'

### Pangeo is:

- a community promoting open, reproducible, and scalable science.
- an integrated ecosystem of open source software tools.
- a community platform for Big Data Geoscience.

### Where to find Pangeo:

- Online: http://pangeo.io/
- GitHub: https://github.com/pangeo-data/
- Discourse: https://discourse.pangeo.io/

<div style="text-align:center"><img src="_static/scientific_python_eco.png" width="100%" /></div>


## The Basics

NumPy / SciPy / Pandas/ Jupyter


### NumPy

NumPy is the fundamental package for scientific computing with Python. It contains among other things:

- a powerful N-dimensional array object
- sophisticated (broadcasting) functions
- tools for integrating C/C++ and Fortran code
- useful linear algebra, Fourier transform, and random number capabilities

In [ ]:
import numpy as np

x = np.ones((4, 2))
x.sum(axis=1)

### SciPy

SciPy is a collection of mathematical algorithms and convenience functions built on the NumPy extension of Python. It contains subpackages that cover:

- clustering
- FFTs
- interpolation
- linear algebra
- singal processing
- stats
- optimization

In [ ]:
from scipy import spatial
x, y = np.mgrid[0:5, 2:8]
tree = spatial.KDTree(list(zip(x.ravel(), y.ravel())))
pts = np.array([[0, 0], [2.1, 2.9]])
tree.query(pts)

tree.query(pts[0])

### Pandas

Pandas is a Python package providing fast, flexible, and expressive data structures designed to make working with “relational” or “labeled” data both easy and intuitive. Pandas is well suited for:

- Tabular data types (see `Series` and `DataFrame` objects)
- Timeseries data manipulation like resampling
- Database-like operations like aligning, merging, joining, reshaping, and grouping

In [ ]:
import pandas as pd

df = pd.read_csv('./data/chico_temperature.csv', index_col=0, parse_dates=True)
display(df.head())
df.resample('AS').max().plot()

## Jupyter

TODO:

- JupyterHub
- Jupyterlab

## Legacy Climate Science Tools

![legacy tools](_static/legacy_tools.png)

## Legacy Tools

### Pros

- "Metadata aware" -- understand the relationships between variables, coordinates, etc.
- "Lazy evaluation" -- don't perform computations until results are actually needed
- Built-in visualization

### Cons

- Hard to scale up, don't handle big data well
- Hard to hack / extend / modify
- Isolated from broader software ecosystem

_Can we have the best of both worlds?_

## Xarray

<!-- <div style="text-align:center"><img src="_static/dataset-diagram.png" width="50%" /></div> -->
<img src="_static/dataset-diagram.png" align="right" width=66% alt="Xarray Dataset">


Xarray is a Python library that provides data structures and tools for working with multidimensional labeled datasets and arrays. Xarray enables users to perform operations on complex datasets making it a powerful high-level tool for data analysis. 

- Inspired by Pandas and NetCDF
- Labeled N-Dimensional data structures (`DataArray` and `Dataset`)
- Toolkit for data manipulation and visualization
- Integrates with scientific Python ecosystem (Pandas/Matplotlib/Dask/etc)
- Backend support for a wide range of ND data formats (NetCDF, GRIB, Raster, Zarr)


#### xarray.Dataset

In [ ]:
import xarray as xr
ds = xr.tutorial.load_dataset('air_temperature')
ds

#### xarray.DataArray

In [ ]:
da = ds['air']

da.isel(time=0, lat=5, lon=10)  # select using integer index
da.sel(time='2013-01-01T18:00:00', lat=62.5, lon=225.0)  # select based on label

In [ ]:
# resample data to monthly means
da_month = da.resample(time='MS').mean('time')

da_climo = da_month.groupby('time.month').mean('time')
da_climo
# remove the monthly climotology
da_no_climo = da_month.groupby('time.month') - da_climo
da_no_climo

In [ ]:
# integrated plotting
da_climo.plot(col='month', col_wrap=6, robust=True)

## Dask

<img src="https://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg" align="right" width=50% alt="Dask Logo">

Dask is a flexible parallel computing library for analytic computing. Dask provides dynamic parallel task scheduling and high-level big-data collections like dask.array and dask.dataframe.

- Dask Array implements a subset of the NumPy ndarray interface using blocked algorithms, cutting up the large array into many small arrays.
- Tools like Xarray and Iris use Dask arrays under-the-hood
- Dask can be deployed on a local computer, HPC, or the Cloud

## Dask Arrays

A dask array looks and feels a lot like a numpy array. However, a dask array doesn't directly hold any data. Instead, it symbolically represents the computations needed to generate the data. Nothing is actually computed until the actual numerical values are needed. This mode of operation is called "lazy"; it allows one to build up complex, large calculations symbolically before turning them over the scheduler for execution.

In [ ]:
# Numpy
import numpy as np
shape = (1000, 4000)
ones_np = np.ones(shape)
ones_np

In [ ]:
# Dask
import dask.array as da
chunk_shape = (1000, 1000)
ones = da.ones(shape, chunks=chunk_shape)
ones

### Deploying Dask

The Dask Schedulers orchestrate the tasks in the Task Graphs so that they can be run in parallel. How they run in parallel, though, is determined by which Scheduler you choose.

There are 3 *local* schedulers:

- **Single-Thread Local**: For debugging, profiling, and diagnosing issues
- **Multi-threaded**: Using the Python built-in threading package (the default for all Dask operations except Bags)
- **Multi-process**: Using the Python built-in multiprocessing package (the default for Dask Bags)

and 1 distributed scheduler, which we will talk about later:

- **Distributed**: Using the dask.distributed module (which uses tornado for TCP communication). The distributed scheduler uses a Cluster to manage communication between the scheduler and the "workers". This is described in the next section.

### Distributed Clusters (http://distributed.dask.org/)¶
Dask can be deployed on distributed infrastructure, such as a an HPC system or a cloud computing system.

- LocalCluster - Creates a Cluster that can be executed locally. Each Cluster includes a Scheduler and Workers.
- Client - Connects to and drives computation on a distributed Cluster

**Dask Jobqueue (http://jobqueue.dask.org/)**
- PBSCluster
- SlurmCluster
- etc.

**Dask Kubernetes (http://kubernetes.dask.org/)**
- KubeCluster

In [ ]:
from dask.distributed import Client
# On Cheyenne:
from dask_jobqueue import PBSCluster
cluster = PBSCluster(project=...)

# On Caseper
from dask_jobqueue import SlurmCluster
cluster = SlurmCluster(project=...)

# On the cloud
from dask_kubernetes import KubeCluster
cluster = KubeCluster()

# use adaptive scaling!
cluster.adapt(0, 30)
client = Client(cluster)

## Visualization

Python a rich ecosystem of data visualization tools.

- Matplotlib:
- Holoviz (Bokeh, Hvplot, Datashader, Panel):
- Cartopy:

More coming here soon.

## Data Catalogs

## Climate-Specific Packages

Everything we just showed was very general purpose.

Now a brief tour of some more specialized tools.

## Principles for Pangeo Packages

- Consume and produce xarray data structures (don't do I/O)
- Operate eagerly on NumPy inputs and lazily on Dask inputs 
- Follow existing metadata standards (e.g. CF convetions)
- Keep it as simple as possible! Solve one problem well

This favors many small, specialized packages.

## xESMF: Universal Regridder for Geospatial Data

<https://xesmf.readthedocs.io/en/latest/>

- **Powerful**: It uses ESMF/ESMPy as backend and can regrid between general curvilinear grids with all ESMF regridding algorithms, such as bilinear, conservative and nearest neighbour.
- **Easy-to-use**: It abstracts away ESMF’s complicated infrastructure and provides a simple, high-level API, compatible with xarray as well as basic numpy arrays.
- **Fast**: It is faster than ESMPy’s original Fortran regridding engine in serial case, and also supports dask for out-of-core, parallel computation.



In [ ]:
import cartopy.crs as ccrs
from matplotlib import pyplot as plt

### Curvilinear Coordinates

In [ ]:
ds = xr.tutorial.load_dataset('rasm')
plt.figure(figsize=(12, 4))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ds.Tair[0].plot(x='xc', y='yc', ax=ax, add_colorbar=False)

### Output Grid

A regular, coarse resolution lat-lon grid.

In [ ]:
import xesmf
ds_out = xesmf.util.grid_global(5, 4)
ds_out

In [ ]:
dsr = ds.rename({'xc': 'lon', 'yc': 'lat'})
regridder = xesmf.Regridder(dsr, ds_out, 'bilinear')
ds_out = regridder(dsr.Tair)

Cell above dies on my macbook with error

    Fatal error in MPI_Init_thread: Other MPI error, error stack:
    MPIR_Init_thread(572)..............: 
    MPID_Init(224).....................: channel initialization failed
    MPIDI_CH3_Init(105)................: 
    MPID_nem_init(324).................: 
    MPID_nem_tcp_init(178).............: 
    MPID_nem_tcp_get_business_card(425): 
    MPID_nem_tcp_init(384).............: gethostbyname failed, Ryans-MacBook-Pro.local (errno 1)
    [unset]: write_line error; fd=-1 buf=:cmd=abort exitcode=3191567
    :

https://github.com/JiaweiZhuang/xESMF/issues/68

## climpred: analysis of ensemble forecast models for climate prediction

<https://climpred.readthedocs.io/en/stable/>

> climpred aims to be the primary package used to analyze output from initialized dynamical forecast models, ranging from short-term weather forecasts to decadal climate forecasts. The code base will be driven entirely by the geoscientific prediction community through open source development. It leverages xarray to keep track of core prediction ensemble dimensions (e.g., ensemble member, initialization date, and lead time) and dask to perform out-of-memory computations on large datasets.

## EOFS: EOF analysis of spatial-temporal data

<https://ajdawson.github.io/eofs/latest/>

- Suitable for large data sets: computationally efficient for the large output data sets of modern climate models.
- Transparent handling of missing values: missing values are removed automatically during computations and placed back into output fields.
- Automatic metadata: metadata from input fields is used to construct metadata for output fields.
- No Compiler required: a fast implementation written in pure Python using the power of numpy, no Fortran or C dependencies.

In [ ]:
from eofs.xarray import Eof
solver = Eof(sst_anom_detrended, weights=np.sqrt(weights))
eof1 = solver.eofsAsCorrelation(neofs=1)
pc1 = solver.pcs(npcs=1, pcscaling=1)
eof1.sel(mode=0).plot()

![eofs result](_static/eofs_output.png)